# *Adversarial Training*
fgsm 공격 후 adversarial examples를 생성하여 훈련하는 과정

In [21]:
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam

In [22]:
# FGSM 공격 함수
def fgsm_attack(model, images, labels, epsilon):
    images = tf.convert_to_tensor(images)  # numpy 배열을 텐서로 변환
    labels = tf.convert_to_tensor(labels)  # 레이블도 텐서로 변환
    with tf.GradientTape() as tape:
        tape.watch(images)
        predictions = model(images)
        loss = tf.keras.losses.sparse_categorical_crossentropy(labels, predictions)
    gradient = tape.gradient(loss, images)
    adversarial_images = images + epsilon * tf.sign(gradient)
    adversarial_images = tf.clip_by_value(adversarial_images, 0, 1)
    return adversarial_images

In [23]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [24]:
# 모델 로드
model = tf.keras.models.load_model('ShipClassifierV1.h5')

In [25]:
# 데이터셋 경로 설정 및 로드
train_dir = '/kaggle/input/ships-dataset/Ships dataset/train'
test_dir = '/kaggle/input/ships-dataset/Ships dataset/test'

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(64, 64),
    batch_size=batch_size,
    class_mode='sparse'
)

# 모델 컴파일 및 학습 설정
batch_size = 32
epochs = 10
epsilon = 0.1  # FGSM 공격에서 사용할 epsilon
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

Found 7406 images belonging to 2 classes.
Found 381 images belonging to 2 classes.


In [26]:
# Adversarial Training 루프
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    
    for batch_images, batch_labels in train_generator:
        # FGSM 공격으로 적대적 예제 생성
        adversarial_images = fgsm_attack(model, batch_images, batch_labels, epsilon)
        
        # 원본 및 적대적 예제 학습
        with tf.GradientTape() as tape:
            predictions = model(batch_images, training=True)
            adv_predictions = model(adversarial_images, training=True)
            loss = tf.keras.losses.sparse_categorical_crossentropy(batch_labels, predictions)
            adv_loss = tf.keras.losses.sparse_categorical_crossentropy(batch_labels, adv_predictions)
            total_loss = loss + adv_loss
        
        gradients = tape.gradient(total_loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        # 가시화: 주기적으로 생성된 적대적 예제를 시각적으로 비교
        if np.random.rand() < 0.1:  # 10% 확률로 가시화
            original_image = batch_images[0] * 255
            adversarial_image = adversarial_images[0].numpy() * 255
            original_image = np.clip(original_image, 0, 255).astype("uint8")
            adversarial_image = np.clip(adversarial_image, 0, 255).astype("uint8")
            
            # 이미지 나란히 보여주기
            combined = np.hstack([original_image, adversarial_image])
            cv2.imshow("Original vs Adversarial", cv2.resize(combined, (768, 384)))
            cv2.waitKey(500)

Epoch 1/10


In [ ]:
# 에포크 종료 시 가시화 창 닫기
cv2.destroyAllWindows()